In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset, DataLoader
from torch import optim
from torch.optim import lr_scheduler
# import torchmetrics
from sklearn.metrics import mean_squared_error, accuracy_score
import numpy as np

import collections
import pandas as pd
import json
from tqdm.auto import tqdm, trange

from datasets import load_metric
import datasets
from transformers import AutoConfig, AutoTokenizer, BertModel, RobertaModel
from transformers import BertForSequenceClassification
from transformers import TrainingArguments, Trainer

import matplotlib.pyplot as plt
import os

In [4]:
# https://github.com/huggingface/transformers/issues/5486
# os.environ["TOKENIZERS_PARALLELISM"] = "false-"
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
with open('../data/xslue/tasks.json', 'r') as f:
    tasks = json.load(f)
tasks

{'CrowdFlower': 13,
 'DailyDialog': 7,
 'EmoBank_Valence': 1,
 'EmoBank_Arousal': 1,
 'EmoBank_Dominance': 1,
 'HateOffensive': 3,
 'PASTEL_age': 8,
 'PASTEL_country': 2,
 'PASTEL_education': 10,
 'PASTEL_ethnic': 10,
 'PASTEL_gender': 3,
 'PASTEL_politics': 3,
 'PASTEL_tod': 5,
 'SARC': 2,
 'SarcasmGhosh': 2,
 'SentiTreeBank': 1,
 'ShortHumor': 2,
 'ShortJokeKaggle': 2,
 'ShortRomance': 2,
 'StanfordPoliteness': 1,
 'TroFi': 2,
 'VUA': 2}

In [7]:
selected_task = ['PASTEL_country',
                 'SARC',
                 'SarcasmGhosh',
                 'ShortHumor',
                 'ShortJokeKaggle',
                 'ShortRomance',
                 'TroFi',
                 'VUA',
                ]

In [8]:
base_model = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(base_model)

In [9]:
tokenizer(['hi, this is joey', 'i want to sleep'], truncation=True, padding=True, max_length=128)

{'input_ids': [[101, 7632, 1010, 2023, 2003, 9558, 102], [101, 1045, 2215, 2000, 3637, 102, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0]]}

In [11]:
singletaskbert = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2) 


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [13]:
singletaskbert.bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
class MyMultitaskDataset(Dataset): 
    # currently it's a Mapping-style dataset. Not sure if a Iterable-style dataset will be better
    def __init__(self, selected_tasks, split, tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")):
        self.tokenizer = tokenizer
        self.tasks = selected_tasks
        self.num_task = len(self.tasks)
        self.num_labels = 0
        self.dfs = []
        self.ranges = [0]
        for task in self.tasks:
            self.num_labels += tasks[task]
            tsv_file = f'../data/xslue/{split}/{task}.tsv'
            df = pd.read_csv(tsv_file, sep='\t')
            df = df.dropna()
            df = df.reset_index(drop=True)
            self.dfs.append(df)
            self.ranges.append(len(df))
        self.ranges = np.cumsum(self.ranges)
        
        
        if self.df['label'].dtype == 'float64':
            self.df['label'] = self.df['label'].astype('float32')
        self.labels = self.df['label'].tolist()
    
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        encodings = tokenizer(self.df['text'].tolist(), truncation=True, padding=True, max_length=128)
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item
